# Exercise 3: Reasoning, Memory, Tool calling examples

In [ ]:
from dotenv import load_dotenv
_ = load_dotenv()

## Reasoning in LLMs

In [ ]:
from langchain_openai import ChatOpenAI

# Reasoning-style model: ask it to think step by step
model = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

question = """
I want a 3‑day trip to Miami in March.
Suggest a rough plan (day 1–3) and explain your reasoning step by step.
"""

response = model.invoke(question)
print(response.content)

## Memory to maintain context

#### Without explicitly adding memory

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

print("=== No memory ===")
print(llm.invoke("I love New York city. Remember it.").content)
print()
print(llm.invoke("Do I love New York city?").content)  # often wrong / generic

#### With memory added

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

print("=== With simple manual memory ===")
context = "I love New York city. Remember that about me.\n"
print(llm.invoke(context + "Say hello to that city.").content)
print()

# Re‑use the same context when asking the next question
print(llm.invoke(context + "Which city do I love?").content)

### Tool use

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain.agents import create_agent

# 1) Define a simple tool
@tool
def get_mock_weather(city: str) -> str:
    """Return fake weather for demo purposes."""
    return f"The weather in {city} is usually warm and sunny this week."

# 2) Build an agent that can call that tool
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

system_prompt = """
You are a helpful travel assistant.
Use the get_mock_weather tool when the user asks about weather.
Explain briefly what you did.
"""

agent = create_agent(
    model=model,
    tools=[get_mock_weather],
    system_prompt=system_prompt,
)

# 3) Ask a question; agent decides if it should call the tool
result = agent.invoke(
    {
        "messages": [
            {"role": "user", "content": "I'm visiting Miami. How's the weather this week?"}
        ]
    }
)

print(result["messages"][-1].content)